In [1]:
import os
import boto3 
import pandas as pd

In [3]:
bucket_name = 'advanced-ml-project' # Add your bucket name
file_name = 'sessions_train.csv'
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket_name, Key=file_name) # S3 uses key-value structure where key is your file name
df_train = pd.read_csv(obj['Body'])
df_train.head()

,Unnamed: 0,session_id,timezone,timezone_offset,previous_sessions_duration,is_user_first_session,is_session,country,region,city,...,locale,os_name,session_index,device_id,user_id_hash,start_dow,start_hour,start_date,create_date,create_session_time
0,0,5558845121177764917,Asia/Manila,28800000.0,25837591,False,True,PH,00,makati,...,en_GB,Android OS,30,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,3,1,2018-11-15,2018-10-07,39
1,1,18781111175537580,Asia/Manila,28800000.0,11343848,False,True,PH,00,makati,...,en_GB,Android OS,10,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,3,7,2018-10-11,2018-10-07,4
2,2,1477540082628742048,Asia/Manila,28800000.0,13499724,False,True,PH,11,davao city,...,en_GB,Android OS,13,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,6,19,2018-10-21,2018-10-07,14
3,3,8184875317380844086,Asia/Manila,28800000.0,32788010,False,True,PH,00,makati,...,en_GB,Android OS,41,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,1,7,2018-11-20,2018-10-07,44
4,4,4706180700083856343,Asia/Manila,28800000.0,5872534,False,True,PH,11,davao city,...,en_GB,Android OS,4,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0,19,2018-10-08,2018-10-07,1


### Time_stamp

* aggregate features - user-level

In [4]:
df_train = df_train.drop(['Unnamed: 0'], axis=1)

In [5]:
df_train.head()

,session_id,timezone,timezone_offset,previous_sessions_duration,is_user_first_session,is_session,country,region,city,latitude,...,locale,os_name,session_index,device_id,user_id_hash,start_dow,start_hour,start_date,create_date,create_session_time
0,5558845121177764917,Asia/Manila,28800000.0,25837591,False,True,PH,00,makati,14.554729,...,en_GB,Android OS,30,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,3,1,2018-11-15,2018-10-07,39
1,18781111175537580,Asia/Manila,28800000.0,11343848,False,True,PH,00,makati,14.554729,...,en_GB,Android OS,10,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,3,7,2018-10-11,2018-10-07,4
2,1477540082628742048,Asia/Manila,28800000.0,13499724,False,True,PH,11,davao city,7.190708,...,en_GB,Android OS,13,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,6,19,2018-10-21,2018-10-07,14
3,8184875317380844086,Asia/Manila,28800000.0,32788010,False,True,PH,00,makati,14.554729,...,en_GB,Android OS,41,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,1,7,2018-11-20,2018-10-07,44
4,4706180700083856343,Asia/Manila,28800000.0,5872534,False,True,PH,11,davao city,7.190708,...,en_GB,Android OS,4,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0,19,2018-10-08,2018-10-07,1


In [6]:
pd.isnull(df_train).any()

session_id                    False
timezone                       True
timezone_offset               False
previous_sessions_duration    False
is_user_first_session         False
is_session                    False
country                        True
region                         True
city                           True
latitude                       True
longitude                      True
locale                         True
os_name                        True
session_index                 False
device_id                      True
user_id_hash                  False
start_dow                     False
start_hour                    False
start_date                    False
create_date                   False
create_session_time           False
dtype: bool

In [9]:
train_feature = pd.DataFrame(df_train.user_id_hash.unique(), columns=['user_id_hash'])

In [11]:
train_feature['most_freq_dow'] = train_feature.user_id_hash.map(df_train.groupby('user_id_hash').start_dow.agg(lambda x: x.value_counts().index[0]))

In [12]:
train_feature['most_freq_hour'] = train_feature.user_id_hash.map(df_train.groupby('user_id_hash').start_hour.agg(lambda x: x.value_counts().index[0]))

In [13]:
train_feature['median_create_session_time'] = train_feature.user_id_hash.map(df_train.groupby('user_id_hash').create_session_time.median())

### session 

* study more on user session properties to get duration, interval features.

In [17]:
train_feature['session_count'] = train_feature.user_id_hash.map(df_train.groupby('user_id_hash').session_index.agg(lambda x: max(x)-min(x)+1))

In [18]:
train_feature['session_durations'] = train_feature.user_id_hash.map(df_train.groupby('user_id_hash').previous_sessions_duration.sum())

In [19]:
train_feature['mean_session_durations'] = train_feature.session_durations/train_feature.session_count

In [20]:
train_feature['mean_session_interval'] = train_feature.median_create_session_time/train_feature.session_count

In [ ]:
train_feature['mean_session_interval'].replace(np.inf, 0, inplace=True)

### geographic

* most frequent country/region/city

In [21]:
df_train.country = df_train.country.fillna('others')
df_train.region = df_train.region.fillna('others')
df_train.city = df_train.city.fillna('others')

In [22]:
train_feature['most_freq_country'] = train_feature.user_id_hash.map(df_train.groupby('user_id_hash').country.agg(lambda x: x.value_counts().index[0]))
train_feature['most_freq_region'] = train_feature.user_id_hash.map(df_train.groupby('user_id_hash').region.agg(lambda x: x.value_counts().index[0]))
train_feature['most_freq_city'] = train_feature.user_id_hash.map(df_train.groupby('user_id_hash').city.agg(lambda x: x.value_counts().index[0]))

In [ ]:
# leave local for now because it contains some error as well as some replicates en-us-us vs en-us

### device info

In [23]:
df_train.os_name = df_train.os_name.fillna('others')
train_feature['most_freq_os'] = train_feature.user_id_hash.map(df_train.groupby('user_id_hash').os_name.agg(lambda x: x.value_counts().index[0]))

In [ ]:
# leave device id right now 

### join events info to get more features

In [ ]:
file_name = 'events.csv'
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket_name, Key=file_name) # S3 uses key-value structure where key is your file name
event = pd.read_csv(obj['Body'])
event.head()

In [29]:
event['label'] = event.event.apply(lambda x: 1 if x=='8' else 0)

In [ ]:
df_event = event.groupby('session_id').label.sum()
df_event = df_event.reset_index()

In [ ]:
purchase_time = event[event.label==1].groupby('session_id').event_timestamp.median()
purchase_time = purchase_time.reset_index()

In [ ]:
# get session level purchase count and purchase time info
df_event2 = pd.merge(df_event,purchase_time, how='left', on=['session_id'])

In [ ]:
# based on session train dataframe join event train, thus we only get info before Dec
df_train2 = pd.merge(df_train,df_event2, how='left', on=['session_id'])

In [ ]:
# user purchase count
train_feature['purchase_counts'] = train_feature.user_id_hash.map(df_train2.groupby('user_id_hash').label.sum())

In [ ]:
df_train2['purchase_time'] = pd.to_datetime(df_train2.event_timestamp+df_train2.timezone_offset, unit='ms')
df_train2['purchase_dow'] = df_train2.median_purchase_time.dt.dayofweek
df_train2['purchase_hour'] = df_train2.median_purchase_time.dt.hour

In [ ]:
# most frequent purchase dow / hour
train_feature['most_frequent_purchase_dow'] = train_feature.user_id_hash.map(df_train2[df_train2.label>0].groupby('user_id_hash').purchase_dow.agg(lambda x: x.value_counts().index[0]))
train_feature.most_frequent_purchase_dow = train_feature.most_frequent_purchase_dow.fillna(7)

In [ ]:
train_feature['most_frequent_purchase_hour'] = train_feature.user_id_hash.map(df_train2[df_train2.label>0].groupby('user_id_hash').purchase_hour.agg(lambda x: x.value_counts().index[0]))
train_feature.most_frequent_purchase_hour = train_feature.most_frequent_purchase_hour.fillna(24)

In [ ]:
train_feature['purchase_perc'] = train_feature.purchase_counts/test_feature.session_count

In [ ]:
# add purchase amount info from event
event_train = pd.read_csv('feature_eng/training_events.csv')

total_amount = event_train.groupby('user_id_hash', as_index=False)['event_value'].\
    sum().rename(columns={'event_value':'total_amount'})

train_feature = pd.merge(train_feature, total_amount, how='left', on='user_id_hash')
train_feature.fillna(0, inplace=True)

### Label encoding

* Use label encoding encode categorical columns

In [ ]:
# fillna by most frequentist value
cols = list(train_feature.columns)
for col in cols:
    train_feature[col] = train_feature[col].fillna(train_feature[col].value_counts().index[0])

In [ ]:
from sklearn.preprocessing import LabelEncoder
cols = ['most_freq_country', 'most_freq_region', 'most_freq_city', 'most_freq_os']
for col in cols:
    encoder = LabelEncoder()
    train_feature[col] = encoder.fit_transform(train_feature[col])

In [10]:
train_feature.head()

,user_id_hash,most_freq_dow,most_freq_hour,median_create_session_time,session_count,session_durations,mean_session_durations,mean_session_interval,most_freq_country,most_freq_region,most_freq_city,most_freq_os,purchase_counts,most_frequent_purchase_dow,most_frequent_purchase_hour,purchase_perc,total_amount
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,6,7,36.0,46,938183565,2.039529e+07,0.782609,162,0,13825,0,1.0,6.0,13.0,0.021739,3.493
1,43f75f8042d3c80c45e222bdd09267f4584684c54d6fae...,5,23,0.0,3,6074816,2.024939e+06,0.000000,69,281,22704,1,0.0,7.0,24.0,0.000000,0.000
2,999524249720812f2d8c0390293efd58e1ac84d587a01c...,2,22,0.0,1,0,0.000000e+00,0.000000,210,299,3615,1,0.0,7.0,24.0,0.000000,0.000
3,4e6bc35cf7fd79a5312047651e7865915f4a6bec193cf2...,3,15,2.0,1,596967,5.969670e+05,2.000000,94,162,8974,0,0.0,7.0,24.0,0.000000,0.000
4,dc009148ee26d658e0240c7b7f6a258790a457737f96e8...,2,2,0.5,1,97482,9.748200e+04,0.500000,11,494,22861,0,0.0,7.0,24.0,0.000000,0.000


In [ ]:
train_feature.to_csv('feature_eng/train_feature.csv')

### Combine train features and labels

In [ ]:
train_label = pd.read_csv('feature_eng/train_label.csv')
train_dataset = pd.merge(training, train_label, how='left', on='user_id_hash')

In [14]:
train_dataset.head()

,user_id_hash,most_freq_dow,most_freq_hour,median_create_session_time,session_count,session_durations,mean_session_durations,mean_session_interval,most_freq_country,most_freq_region,most_freq_city,most_freq_os,purchase_counts,most_frequent_purchase_dow,most_frequent_purchase_hour,purchase_perc,total_amount,user_purchase_binary_7_days,user_purchase_binary_14_days
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,6,7,36.0,46,938183565,2.039529e+07,0.782609,162,0,13825,0,1.0,6.0,13.0,0.021739,3.493,0.0,0.0
1,43f75f8042d3c80c45e222bdd09267f4584684c54d6fae...,5,23,0.0,3,6074816,2.024939e+06,0.000000,69,281,22704,1,0.0,7.0,24.0,0.000000,0.000,0.0,0.0
2,999524249720812f2d8c0390293efd58e1ac84d587a01c...,2,22,0.0,1,0,0.000000e+00,0.000000,210,299,3615,1,0.0,7.0,24.0,0.000000,0.000,0.0,0.0
3,4e6bc35cf7fd79a5312047651e7865915f4a6bec193cf2...,3,15,2.0,1,596967,5.969670e+05,2.000000,94,162,8974,0,0.0,7.0,24.0,0.000000,0.000,0.0,0.0
4,dc009148ee26d658e0240c7b7f6a258790a457737f96e8...,2,2,0.5,1,97482,9.748200e+04,0.500000,11,494,22861,0,0.0,7.0,24.0,0.000000,0.000,0.0,0.0


In [ ]:
train_dataset.to_csv('feature_eng/train_dataset.csv', index=False)